# Plots for the "Illustrative" Section

In [ ]:
using QAOA, Distributions, Interpolations, LinearAlgebra
using HDF5, Printf
using PythonPlot
PythonPlot.matplotlib.style.use("./paper.mplstyle")
PythonPlot.rc("axes", prop_cycle = PythonPlot.matplotlib.cycler(color = ["#2D5FAA", "#B7293F", "#438E6A", "#F7BD2E", "#F16C37"]))

# PATH = "../";
PATH = "/home/ubuntu/Archives/"
PLOT_PATH = "/home/ubuntu/Archives/plots/SK_model/paper/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)

In [ ]:
N = 9
N = 17

# folder_name = PATH * @sprintf("data/")
subdir = "small_gaps"
# subdir = "large_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name);

In [ ]:
# N = 9
# seed = 22824 # highest peak
# seed = 28563 # second-highest, solved by mean-field

seed = 23583 # clean coupling to first excited state, great illustration!

# large
# seed = 100210

# N = 17
seed = 10173
seed = 16173

In [ ]:
# # look what's inside
# h5open(folder_name * @sprintf("results_random_SK_instance_N_%i_seed_%i.h5", N, seed))

In [ ]:
J_mat = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "J")
mf_problem = Problem(0, J_mat);

In [ ]:
T_final = 32768.
npts = 2048
# npts = 4096
coarse_times = range(0, 1, npts + 1);
exact_times = range(0, 1, 33);

## Exact gap

In [ ]:
λ = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "exact_ARPACK_LM_eigvals");

In [ ]:
gap = λ[2, :] .- λ[1, :];
mingap = minimum(gap)
mingap |> println
gap_idx = findfirst(x -> x == mingap, gap)
gaploc = exact_times[gap_idx]
gaploc |> println

In [ ]:
tol = 1e-8
T_final = 32000.
bogo_spec = h5read(folder_name * @sprintf("results_random_SK_instance_N_%i_seed_%i.h5", N, seed), 
                   @sprintf("bogoliubov_spectrum_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), 32));

In [ ]:
all_eigvecs = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "exact_ARPACK_LM_lowest_eigvecs");

In [ ]:
H_x = SpinFluctuations.hamiltonian(1, 0, mf_problem.local_fields, mf_problem.couplings)
H_z = SpinFluctuations.hamiltonian(0, 1, mf_problem.local_fields, mf_problem.couplings);

In [ ]:
gs = [all_eigvecs[k, :, 1] for k in 1:length(exact_times)]
first_ex = [all_eigvecs[k, :, 2] for k in 1:length(exact_times)]
second_ex = [all_eigvecs[k, :, 3] for k in 1:length(exact_times)]

overlap_01_x = [first_ex[k]' * H_x * gs[k] for k in 1:length(exact_times)]
overlap_01_z = [first_ex[k]' * H_z * gs[k] for k in 1:length(exact_times)]

overlap_02_x = [second_ex[k]' * H_x * gs[k] for k in 1:length(exact_times)]
overlap_02_z = [second_ex[k]' * H_z * gs[k] for k in 1:length(exact_times)];

In [ ]:
figure(figsize = (4, 4))

ax = subplot(211)
for i in 1:14
	if i == 2 || i == 3
		plot(exact_times, (λ[i, :] .- λ[1, :]), lw = 1.5, alpha = 1)
	else
		plot(exact_times, (λ[i, :] .- λ[1, :]), "-k", lw=0.75, alpha = 1)
	end
	plot(exact_times[2:end], bogo_spec[i, :], "--k", lw = 1.5, alpha=0.5)
end
plot([], [], "-k", lw = 1, label = "Exact")
plot([], [], label = "Bogoliubov", "--k", lw=1.5, alpha = 0.5)
xlim(0.0, 1.0)
ax.set_yticks(2 .* [0, 0.5, 1.0])
ylim(-0.0, 2)
ax.set_xticklabels([])
ylabel("\$E_\\alpha - E_0\$")
legend(frameon = false, handlelength=1., handletextpad=0.3, borderaxespad=0.2)

ax = subplot(212)
plot(exact_times, abs.(overlap_02_z .- overlap_02_x), label = "\$\\alpha = 2\$", "-C1", lw=2)
plot(exact_times, abs.(overlap_01_z .- overlap_01_x), label = "\$\\alpha = 1\$", "-C0")
xlim(0.0, 1.0)
ax.set_yticks([0, 1.0])
ylim(0, 1)
xlabel("\$s\$")
ylabel("\${|\\langle \\alpha|H_Z-H_X|0\\rangle|}\$")

legend(frameon = false)

tight_layout(pad=0.25)
# savefig(PLOT_PATH * @sprintf("exact_spectrum_N_%i_seed_%i.pdf", N, seed), dpi=256, bbox_inches="tight")
display(gcf())
PythonPlot.close();

## Mean-field trajectories

In [ ]:
T_final = 32768.
tol = 1e-6
sol_t = h5read(folder_name * @sprintf("results_random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/times", T_final, log10(tol)))
sol_u = h5read(folder_name * @sprintf("results_random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/trajectories", T_final, log10(tol)));

In [ ]:
size(sol_u)

### Bloch coordinates, Magnetization & Complex Coordinates

In [ ]:
nx_vals = n_vals("x", sol_u)
ny_vals = n_vals("y", sol_u)
nz_vals = n_vals("z", sol_u);

In [ ]:
nx_coarse = n_coarse(nx_vals, sol_t, coarse_times)
ny_coarse = n_coarse(ny_vals, sol_t, coarse_times)
nz_coarse = n_coarse(nz_vals, sol_t, coarse_times);

In [ ]:
S_vals = [transpose(reduce(hcat, [nx_coarse[:, k], ny_coarse[:, k], nz_coarse[:, k]])) |> Matrix for k in 1:npts+1]
magnetizations = reduce(hcat, map(S -> magnetization(S, mf_problem.local_fields, mf_problem.couplings), S_vals));

In [ ]:
areas = Dict()
EA_param = Dict()
# dts = [(x[2] - x[1]) / T_final for x in zip(sol_t[1:end-1], sol_t[2:end])]
dts = [(x[2] - x[1]) / T_final for x in zip(coarse_times[1:end-1], coarse_times[2:end])]
for spin_idx in 1:N-1
	# areas[spin_idx] = sum(dts .* nz_vals[spin_idx, 2:end]) |> abs
	areas[spin_idx] = sum(dts .* magnetizations[spin_idx, 2:end]) |> abs

	# Edwards-Anderson
	EA_param[spin_idx] = (1/(N-1)) * sum(dts .* nz_coarse[spin_idx, 2:end] .^ 2) |> abs
end

top_idxs = [k for (k, v) in sort(areas |> collect, by = x -> x[2])]
top_idx = top_idxs[2]

In [ ]:
[(spin_nr, 1e4A) for (spin_nr, A) in EA_param]

In [ ]:
[(spin_nr, 1e4A) for (spin_nr, A) in areas]

In [ ]:
regular_trajectories = filter!(x -> x != top_idx, collect(1:N-1));
regular_trajectories = filter!(x -> x != top_idxs[3], regular_trajectories);

In [ ]:
figure(figsize = (4, 5))

c1 = "-C1"
c2 = "-C0"

ax = subplot(411)
for spin_nr in regular_trajectories
	plot(sol_t ./ sol_t[end], nx_vals[spin_nr, :], "-k", lw=0.75)
end
plot(sol_t ./ sol_t[end], nx_vals[top_idxs[3], :], c2, lw=1, label = @sprintf("\$i=%i\$", top_idxs[3]))
plot(sol_t ./ sol_t[end], nx_vals[top_idx, :], c1, lw=1.5, label = @sprintf("\$i=%i\$", top_idx))
xlim(0, 1)
ax.set_xticklabels([])
ylim(-0.1, 1.1)
ylabel("\$n_i^x(s)\$")
legend(frameon = false)

ax = subplot(412)
for spin_nr in regular_trajectories
	plot(sol_t ./ T_final, nz_vals[spin_nr, :], "-k", lw=0.75)
end
plot(sol_t ./ T_final, nz_vals[top_idxs[3], :], c2, lw=1)
plot(sol_t ./ T_final, nz_vals[top_idx, :], c1, lw=1.5)

# Edwards-Anderson
plot(coarse_times, sum([nz_coarse[spin_nr, :] .^ 2 for spin_nr in 1:N-1]) ./ (N-1), "-C2", lw=2.5, label="\$q_{\\mathrm{EA}}\$")
xlim(0, 1)
ylim(-1.1, 1.1)
ax.set_xticklabels([])
ylabel("\$n_i^z(s)\$")
legend(frameon = false)

ax = subplot(413)
for spin_nr in regular_trajectories
	plot(coarse_times, (abs.(complex_coordinate(spin_nr, nx_coarse, ny_coarse, nz_coarse))), "-k", lw=0.75)
end
plot(coarse_times, (abs.(complex_coordinate(top_idxs[3], nx_coarse, ny_coarse, nz_coarse))), c2, lw=1, label = @sprintf("\$i=%i\$", top_idxs[3]))
plot(coarse_times, (abs.(complex_coordinate(top_idx, nx_coarse, ny_coarse, nz_coarse))), c1, lw=1.5, label = @sprintf("\$i=%i\$", top_idx))

xlim(0, 1)
ylim(0, 1.5)
ax.set_xticklabels([])
ylabel("\$|z_i(s)|\$")

ax = subplot(414)

for spin_nr in regular_trajectories
	plot(coarse_times, magnetizations[spin_nr, :], "-k", lw=0.75)
end
plot(coarse_times, magnetizations[top_idxs[3], :], c2, lw=1, label = @sprintf("\$i=%i\$", top_idxs[3]))
plot(coarse_times, magnetizations[top_idx, :], c1, lw=1.5, label = @sprintf("\$i=%i\$", top_idx))

# for spin_nr in regular_trajectories
# 	plot(coarse_times, magnetizations[spin_nr, :] ./ mf_problem.local_fields[spin_nr], "-k", lw=0.75)
# end
# plot(coarse_times, magnetizations[top_idxs[3], :] ./ mf_problem.local_fields[top_idxs[3]], c2, lw=1, label = @sprintf("\$i=%i\$", top_idxs[3]))
# plot(coarse_times, magnetizations[top_idx, :] ./ mf_problem.local_fields[top_idx], c1, lw=1.5, label = @sprintf("\$i=%i\$", top_idx))
xlim(0.0, 1)
ylim(-3, 3)
xlabel("\$s\$")
ylabel("\$m_i(s)\$")

tight_layout(pad=0.1)
display(gcf())
# savefig(PLOT_PATH * @sprintf("mean_fields_N_%i_seed_%i.pdf", N, seed), dpi=256, bbox_inches="tight")
PythonPlot.close();

## Energies

In [ ]:
sigma_star = sign.(sol_u[end, 3, :])
h = mf_problem.local_fields
J = mf_problem.couplings
E = sum([-h[l] * sigma_star[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star[i] * sigma_star[j] for i in 1:N-1 for j in (i+1):N-1])

In [ ]:
[λ[k, :][end] for k in 1:6] |> println

In [ ]:
isapprox.([λ[k, :][end] for k in 1:6], E, atol=1e-5) |> println

In [ ]:
top_idxs

In [ ]:
for spin_idx in 1:N-1
	sigma_star_flip = sign.(sol_u[end, 3, :])
	sigma_star_flip[spin_idx] = (-1) * sigma_star_flip[spin_idx]
	E_flip = sum([-h[l] * sigma_star_flip[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star_flip[i] * sigma_star_flip[j] for i in 1:N-1 for j in (i+1):N-1])
	println(spin_idx, ": ", E_flip)
end

## Statistical Green function

In [ ]:
T_final = 32000
tol = 1e-8
all_flucs = h5read(folder_name * @sprintf("results_random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("fluctuations_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(1e-8), npts));

In [ ]:
regular_trajectories = filter!(x -> x != top_idxs[1], collect(1:N-1));
regular_trajectories = filter!(x -> x != top_idxs[3], regular_trajectories);

scale_factors = [1 .+ abs.(complex_coordinate(i, nx_coarse, ny_coarse, nz_coarse)) .^ 2 for i in 1:N-1]
mean_flucs = mean([smoothen(all_flucs[i, :], coarse_times) for i in regular_trajectories], dims=1)[1];
mean_scaled_flucs = mean([smoothen(scale_factors[i] .^ 2 .* all_flucs[i, :], coarse_times) for i in regular_trajectories], dims = 1)[1];

In [ ]:
inv_mag = 1 ./ (magnetizations[top_idx, :] .|> abs)
inner_navg = 64
outer_navg = 64
# inv_mag = smoothen(smoothen(magnetizations[top_idx, :], coarse_times, navg=inner_navg) .|> abs, coarse_times, navg=outer_navg)
scale_factor = 1 .+ abs.(complex_coordinate(top_idx, nx_coarse, ny_coarse, nz_coarse)) .^ 2
crit_flucs = smoothen(scale_factor.^2 .* all_flucs[top_idx, :], coarse_times, navg=256) .* inv_mag;

In [ ]:
figure(figsize = (4, 4))
ax = subplot(211)
axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=5)
semilogy(exact_times, abs.(overlap_01_z .- overlap_01_x) ./ ((λ[2, :] .- λ[1, :])), label = "\$\\alpha = 1\$")
semilogy(exact_times, abs.(overlap_02_z .- overlap_02_x) ./ ((λ[3, :] .- λ[1, :])), label="\$\\alpha = 2\$", lw=2)
xlim(0.0, 1.0)
ylim(1e-1, 1e3)
ax.set_xticklabels([])
# ylabel("\$\\frac{|\\langle \\alpha|H_Z-H_X|0\\rangle|}{E_\\alpha - E_0}\$")
ylabel("\$|\\langle \\alpha|H_Z \\hspace{-1mm} - \\hspace{-0.5mm} H_X|0\\rangle|/(E_\\alpha \\hspace{-1mm} - \\hspace{-0.5mm} E_0)\$", fontsize=11)
legend(frameon = false)

ax = subplot(212)
axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=5)
plot(coarse_times, crit_flucs, c1, lw=1.5, label=@sprintf("\$i_*=%s\$", top_idx))
xlim(0, 1)
# ylim(0, 5e1)
# ax.set_yticks([0, 25, 50])
xlabel("\$s\$")
ylabel("\$ \\langle\\delta  z_{i_*}(s) \\delta \\bar z_{i_*}(s) \\rangle / |m_{i_*}(s)|\$", fontsize=12)
legend(frameon = false)

tight_layout(pad=0.2)
# savefig(PLOT_PATH * @sprintf("metric_N_%i_seed_%i.pdf", N, seed), dpi=256, bbox_inches="tight")
display(gcf())
PythonPlot.close();

## Fluctuations & Spectrum

In [ ]:
figure(figsize = (7.5, 2.5))
ax = subplot(121)
all_colors = vcat(["-k", c2, c1], ["-k" for _ in 1:5])
all_labels = vcat(["", @sprintf("\$i=%s\$", string(2)), @sprintf("\$i=%s\$", string(3))], ["" for _ in 1:5])
for i in 1:(N-1)
	scale_factor = 1 # .+ abs.(complex_coordinate(i, nx_coarse, ny_coarse, nz_coarse)) .^ 2
	plot(coarse_times, scale_factor .^ 2 .* smoothen(all_flucs[i, :], coarse_times))#, label=all_labels[i], all_colors[i])
	xlim(0.0, 1.0)
	# legend(frameon = false, handlelength = 1, ncol=1)
end
xlabel("\$s\$")
ax.set_ylabel("\$ -\\mathrm{Im}\\, G^<_{ii}(s, s) \$")
xlim(0, 1)
ylim(0, 0.5)
yticks([0.1k for k in 0:5])
# ax.set_ylabel("\$ \\langle\\delta \\bar z_i(s) \\delta z_i(s) \\rangle\$")

# ax = subplot(122)
# # for k in [t_idx]
# # 	plot(2pi .* ωs[k], spec_sum[k], "-k")
# # end
# axvline.(bogo_spec[N:end, 16+t_idx], c="k", alpha = 0.5, ls = "--", lw=2.5)
# xlim(0., 1.)
# ylim(-0.0, 1)
# yticks([0, 0.5, 1.0])
# xlabel("\$\\omega\$")
# ylabel("\$\\left|\\mathrm{Tr}\\;\\boldsymbol{\\rho}(T, \\omega)\\right|\$", labelpad=8)
# ax.yaxis.set_label_position("right")
# # ax.yaxis.tick_right()

tight_layout(pad=0.1)
# savefig(PLOT_PATH * @sprintf("statistical_and_spectral_N_%i_seed_%i.pdf", N, seed), dpi=256, bbox_inches="tight")
display(gcf())
PythonPlot.close();

## More Details

In [ ]:
figure(figsize = (8, 3))
ylims = (0, 0.4)

for i in 1:(N-1)÷2
	ax = subplot(2, (N - 1) ÷ 2, i)
	c = "k"
	if i in [top_idx]
		c = "C1"
	end
	scale_factor = 1 #.+ abs.(complex_coordinate(i, nx_coarse, ny_coarse, nz_coarse)) .^ 2
	plot(coarse_times, scale_factor .^ 2 .* smoothen(all_flucs[i, :], coarse_times), label = @sprintf("\$i=%s\$", string(i)), c = c)
	xlim(0.0, 1.0)
	ax.set_xticklabels([])
	ylim(ylims...)
	if i > 1
		ax.set_yticklabels([])
	end
	legend(frameon = false, handlelength = 0)
end

ax = subplot(2, (N - 1) ÷ 2, 1)
ax.set_ylabel("\$ \\langle\\delta \\bar z_i(s) \\delta z_i(s) \\rangle\$")

for i in (N-1)÷2+1:N-1
	ax = subplot(2, (N - 1) ÷ 2, i)
	c = "k"
	if i in [top_idx]
		c = "C1"
	end
	scale_factor = 1 #.+ abs.(complex_coordinate(i, nx_coarse, ny_coarse, nz_coarse)) .^ 2
	plot(coarse_times, scale_factor .^ 2 .* smoothen(all_flucs[i, :], coarse_times), label = @sprintf("\$i=%s\$", string(i)), c = c)
	xlim(0.0, 1.0)
	ylim(ylims...)
	if i > (N - 1) ÷ 2 + 1
		ax.set_yticklabels([])
	end
	xlabel("\$s\$")
	legend(frameon = false, handlelength = 0)
end

ax = subplot(2, (N - 1) ÷ 2, (N - 1) ÷ 2 + 1)
ax.set_ylabel("\$ \\langle\\delta \\bar z_i(s) \\delta z_i(s) \\rangle\$")

tight_layout(pad=0.1)
display(gcf())
PythonPlot.close();

In [ ]:
tol = 1e-8
T_final = 32768.0
npts = 4096
coarse_times_fine = range(0, 1, npts + 1);
all_flucs_fine = h5read(folder_name * @sprintf("results_random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("fluctuations_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(1e-8), npts));

In [ ]:
figure(figsize = (4, 3))
plot(coarse_times_fine, mean(all_flucs_fine, dims = 1)[1, :])
plot(coarse_times, smoothen(mean(all_flucs, dims = 1)[1, :], coarse_times), "-")
tight_layout()
xlim(0, 1)
ylim(0)
display(gcf())
tight_layout()
PythonPlot.close();
